In [269]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from numpy.linalg import inv
from numpy.linalg import svd
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
df_raw = pd.read_csv('./CollegeScorecard_Raw_Data_03142022/Most-Recent-Cohorts-Field-of-Study.csv', low_memory = False)

df_raw1 = pd.read_csv('./CollegeScorecard_Raw_Data_03142022/Most-Recent-Cohorts-Institution.csv', low_memory = False)
df_raw=df_raw.replace('PrivacySuppressed', np.NaN)

df_raw.shape

(224849, 121)

In [293]:
df_raw['SOURCE'].value_counts()

KeyError: 'SOURCE'

In [270]:
columns_to_keep = ['UNITID', 'OPEID6', 'INSTNM', 'MAIN', 'CIPCODE', 'CIPDESC', 'CREDLEV']
for i in columns_to_keep:
    df_raw[i]

In [294]:
df_raw1.keys()

Index(['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP',
       'ACCREDAGENCY', 'INSTURL', 'NPCURL',
       ...
       'COUNT_WNE_MALE1_P8', 'MD_EARN_WNE_MALE1_P8', 'GT_THRESHOLD_P10',
       'MD_EARN_WNE_INC1_P10', 'MD_EARN_WNE_INC2_P10', 'MD_EARN_WNE_INC3_P10',
       'MD_EARN_WNE_INDEP1_P10', 'MD_EARN_WNE_INDEP0_P10',
       'MD_EARN_WNE_MALE0_P10', 'MD_EARN_WNE_MALE1_P10'],
      dtype='object', length=2989)

In [274]:
df_raw.keys()

Index(['UNITID', 'OPEID6', 'INSTNM', 'CONTROL', 'MAIN', 'CIPCODE', 'CIPDESC',
       'CREDLEV', 'CREDDESC', 'IPEDSCOUNT1',
       ...
       'EARN_COUNT_WNE_3YR', 'EARN_CNTOVER150_3YR', 'EARN_COUNT_PELL_NE_3YR',
       'EARN_PELL_NE_MDN_3YR', 'EARN_COUNT_NOPELL_NE_3YR',
       'EARN_NOPELL_NE_MDN_3YR', 'EARN_COUNT_MALE_NE_3YR',
       'EARN_MALE_NE_MDN_3YR', 'EARN_COUNT_NOMALE_NE_3YR',
       'EARN_NOMALE_NE_MDN_3YR'],
      dtype='object', length=121)

In [280]:
# # Drop columns which contain all NaN values
# df_raw2 = df_raw.dropna(axis=1, how='all')
df_raw2 = df_raw1.dropna(thresh=1000,axis=1)
# for col in df_raw2.keys():
#     na_num = df_raw2[col].isna().sum()
#     print(na_num)
df_raw2.shape
# df_raw2.loc[:,'CITY']
# df_raw2.columns.get_loc('CITY')

(6694, 2893)

In [281]:
SAT_cols = []
for name in df_new.columns:
    if 'SAT' in name:
        SAT_cols.append(name)

ACT_cols = []
for name in df_new.columns:
    if 'ACT' in name:
        ACT_cols.append(name)

for name in SAT_cols+ACT_cols:
    print(name)
    print(df_raw2[name].isna().sum())

SATVR25
5476
SATVR75
5476
SATMT25
5476
SATMT75
5476
SATVRMID
5476
SATMTMID
5476
SAT_AVG
5416
SAT_AVG_ALL
5266
ACTCM25
5442
ACTCM75
5442
ACTEN25
5524
ACTEN75
5524
ACTMT25
5524
ACTMT75
5524
ACTCMMID
5442
ACTENMID
5524
ACTMTMID
5524


In [282]:
numeric_bool = ((df_raw2.dtypes=='int64') + (df_raw2.dtypes=='float64'))
numeric_col = list(numeric_bool.index[numeric_bool])
df_raw2_numeric = df_raw2[numeric_col]
df_raw2_numeric.head()

,UNITID,OPEID,OPEID6,SCH_DEG,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,ST_FIPS,REGION,LOCALE,LATITUDE,LONGITUDE,CCBASIC,CCUGPROF,CCSIZSET,HBCU,PBI,ANNHI,TRIBAL,AANAPII,HSI,NANTI,MENONLY,WOMENONLY,ADM_RATE,ADM_RATE_ALL,SATVR25,SATVR75,SATMT25,SATMT75,SATVRMID,SATMTMID,ACTCM25,ACTCM75,ACTEN25,ACTEN75,ACTMT25,ACTMT75,ACTCMMID,ACTENMID,ACTMTMID,SAT_AVG,SAT_AVG_ALL,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,PCIP12,PCIP13,PCIP14,PCIP15,PCIP16,PCIP19,PCIP22,PCIP23,PCIP24,PCIP25,PCIP26,PCIP27,PCIP29,PCIP30,PCIP31,PCIP38,PCIP39,PCIP40,PCIP41,PCIP42,PCIP43,PCIP44,PCIP45,PCIP46,PCIP47,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,CIP01CERT1,CIP01CERT2,CIP01ASSOC,CIP01CERT4,CIP01BACHL,CIP03CERT1,CIP03CERT2,CIP03ASSOC,CIP03CERT4,CIP03BACHL,CIP04CERT1,CIP04CERT2,CIP04ASSOC,CIP04CERT4,CIP04BACHL,CIP05CERT1,CIP05CERT2,CIP05ASSOC,CIP05CERT4,CIP05BACHL,CIP09CERT1,CIP09CERT2,CIP09ASSOC,CIP09CERT4,CIP09BACHL,CIP10CERT1,CIP10CERT2,CIP10ASSOC,CIP10CERT4,CIP10BACHL,CIP11CERT1,CIP11CERT2,CIP11ASSOC,CIP11CERT4,CIP11BACHL,CIP12CERT1,CIP12CERT2,CIP12ASSOC,CIP12CERT4,CIP12BACHL,CIP13CERT1,CIP13CERT2,CIP13ASSOC,CIP13CERT4,CIP13BACHL,CIP14CERT1,CIP14CERT2,CIP14ASSOC,CIP14CERT4,CIP14BACHL,CIP15CERT1,CIP15CERT2,CIP15ASSOC,CIP15CERT4,CIP15BACHL,CIP16CERT1,CIP16CERT2,CIP16ASSOC,CIP16CERT4,CIP16BACHL,CIP19CERT1,CIP19CERT2,CIP19ASSOC,CIP19CERT4,CIP19BACHL,CIP22CERT1,CIP22CERT2,CIP22ASSOC,CIP22CERT4,CIP22BACHL,CIP23CERT1,CIP23CERT2,CIP23ASSOC,CIP23CERT4,CIP23BACHL,CIP24CERT1,CIP24CERT2,CIP24ASSOC,CIP24CERT4,CIP24BACHL,CIP25CERT1,CIP25CERT2,CIP25ASSOC,CIP25CERT4,CIP25BACHL,CIP26CERT1,CIP26CERT2,CIP26ASSOC,CIP26CERT4,CIP26BACHL,CIP27CERT1,CIP27CERT2,CIP27ASSOC,CIP27CERT4,CIP27BACHL,CIP29CERT1,CIP29CERT2,CIP29ASSOC,CIP29CERT4,CIP29BACHL,CIP30CERT1,CIP30CERT2,CIP30ASSOC,CIP30CERT4,CIP30BACHL,CIP31CERT1,CIP31CERT2,CIP31ASSOC,CIP31CERT4,CIP31BACHL,CIP38CERT1,CIP38CERT2,CIP38ASSOC,CIP38CERT4,CIP38BACHL,CIP39CERT1,CIP39CERT2,CIP39ASSOC,CIP39CERT4,CIP39BACHL,CIP40CERT1,CIP40CERT2,CIP40ASSOC,CIP40CERT4,CIP40BACHL,CIP41CERT1,CIP41CERT2,CIP41ASSOC,CIP41CERT4,CIP41BACHL,CIP42CERT1,CIP42CERT2,CIP42ASSOC,CIP42CERT4,CIP42BACHL,CIP43CERT1,CIP43CERT2,CIP43ASSOC,CIP43CERT4,CIP43BACHL,CIP44CERT1,CIP44CERT2,CIP44ASSOC,CIP44CERT4,CIP44BACHL,CIP45CERT1,CIP45CERT2,CIP45ASSOC,CIP45CERT4,CIP45BACHL,CIP46CERT1,CIP46CERT2,CIP46ASSOC,CIP46CERT4,CIP46BACHL,CIP47CERT1,CIP47CERT2,CIP47ASSOC,CIP47CERT4,CIP47BACHL,CIP48CERT1,CIP48CERT2,CIP48ASSOC,CIP48CERT4,CIP48BACHL,CIP49CERT1,CIP49CERT2,CIP49ASSOC,CIP49CERT4,CIP49BACHL,CIP50CERT1,CIP50CERT2,CIP50ASSOC,CIP50CERT4,CIP50BACHL,CIP51CERT1,CIP51CERT2,CIP51ASSOC,CIP51CERT4,CIP51BACHL,CIP52CERT1,CIP52CERT2,CIP52ASSOC,CIP52CERT4,CIP52BACHL,CIP54CERT1,CIP54CERT2,CIP54ASSOC,CIP54CERT4,CIP54BACHL,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,NPT4_PUB,NPT4_PRIV,NPT41_PUB,NPT42_PUB,NPT43_PUB,NPT44_PUB,NPT45_PUB,NPT41_PRIV,NPT42_PRIV,NPT43_PRIV,NPT44_PRIV,NPT45_PRIV,NPT4_048_PUB,NPT4_048_PRIV,NPT4_3075_PUB,NPT4_3075_PRIV,NPT4_75UP_PUB,NPT4_75UP_PRIV,NUM4_PUB,NUM4_PRIV,NUM41_PUB,NUM42_PUB,NUM43_PUB,NUM44_PUB,NUM45_PUB,NUM41_PRIV,NUM42_PRIV,NUM43_PRIV,NUM44_PRIV,NUM45_PRIV,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PFTFAC,PCTPELL,C150_4,C150_L4,C150_4_POOLED,C150_L4_POOLED,POOLYRS,PFTFTUG1_EF,D150_4,D150_L4,D150_4_POOLED,D150_L4_POOLED,C150_4_WHITE,C150_4_BLACK,C150_4_HISP,C150_4_ASIAN,C150_4_AIAN,C150_4_2MOR,C150_4_NRA,C150_4_UNKN,C150_L4_WHITE,C150_L4_BLACK,C150_L4_HISP,C150_L4_ASIAN,C150_L4_2MOR,C150_L4_UNKN,C200_4,C200_L4,D200_4,D200_L4,RET_FT4,RET_FTL4,RET_PT4,RET_PTL4,C200_4_POOLED,C200_L4_POOLED,POOLYRS200,D200_4_POOLED,D200_L4_POOLED,PCTFLOAN,UG25ABV,CDR2,CDR3,COUNT_WNE_P10,MD_EARN_WNE_P10,PCT25_EARN_WNE_P10,PCT75_EARN_WNE_P10,COUNT_WNE_INC1_P10,COUNT_WNE_INC2_P10,COUNT_WNE_INC3_P10,COUNT_WNE_INDEP0_P10,COUNT_WNE_INDEP1_P10,COUNT_WNE_MALE0_P10,COUNT_WNE_MALE1_P10,COUNT_WNE_P6,MD_EARN_WNE_P6,PCT25_EARN_WNE_P6,PCT75_EARN_WN

In [283]:
df_raw2_numeric[['INSTNM',"CITY", "STABBR"]] = df_raw2[['INSTNM',"CITY", "STABBR"]]
df_new = df_raw2_numeric
df_new=df_new.drop(["OPEID",'OPEID6','MAIN','INEXPFTE', 'ICLEVEL', 'OPEFLAG', 'UNITID'],axis = 1)
df_new.head()
df_new.shape

(6694, 718)

In [217]:
df_new.keys()

Index(['HCM2', 'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'ST_FIPS',
       'REGION', 'LOCALE', 'LATITUDE', 'LONGITUDE',
       ...
       'POOLYRSOM_PELL_FTNFT', 'POOLYRSOM_PELL_PTNFT', 'POOLYRSOM_PELL_ALL',
       'POOLYRSOM_PELL_FIRSTTIME', 'POOLYRSOM_PELL_NOTFIRSTTIME',
       'POOLYRSOM_PELL_FULLTIME', 'POOLYRSOM_PELL_PARTTIME', 'INSTNM', 'CITY',
       'STABBR'],
      dtype='object', length=652)

In [284]:
df_new = df_new.fillna(df_new.mean())

## See
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html

In [285]:
import numpy as np

feat_names = [name for name in df_new.columns if name not in SAT_cols+ACT_cols+['INSTNM',"CITY", "STABBR"]]
X = np.array(df_new[feat_names])

## feature standardization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

## feature selection
from sklearn.feature_selection import VarianceThreshold
threshold = 0.01
selector = VarianceThreshold(threshold = threshold)
X_varsel = selector.fit_transform(X_scaled)

y = np.array(df_new['SAT_AVG_ALL'])

from sklearn.model_selection import train_test_split

X_train_all, X_test, y_train_all, y_test = train_test_split(X_varsel, y, test_size=0.20, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, test_size=0.20, random_state=42)

In [286]:
ind_sel = np.where(np.array([np.var(X_scaled[:,j]) for j in range(X_scaled.shape[1])])>=threshold)[0]
feat_sel = [feat_names[k] for k in ind_sel]

In [287]:
X_varsel.shape

(6694, 326)

In [288]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(4284, 326)
(1071, 326)
(1339, 326)


In [289]:
from sklearn.linear_model import Ridge
import numpy as np

for alp in np.linspace(4,5,11):
    print(f'alpha: {alp}')
    reg = Ridge(alpha=alp)
    reg.fit(X_train, y_train)

#     mse_val = np.mean((reg.predict(X_val)-y_val)**2)
#     mse_train = np.mean((reg.predict(X_train)-y_train)**2)
    #mse_test = np.mean((reg.predict(X_test)-y_test)**2)

    R2_train = reg.score(X_train, y_train)
    R2_val = reg.score(X_val, y_val)
    #R2_test = reg.score(X_test, y_test)
    print('R2_train: ', R2_train)
    print('R2_val: ', R2_val)
    #print('R2_test: ', R2_test)
    print('\n')

alpha: 4.0
R2_train:  0.6069878199995933
R2_val:  0.5006403343085457


alpha: 4.1
R2_train:  0.6067742973445586
R2_val:  0.5005403508220736


alpha: 4.2
R2_train:  0.6065631162261185
R2_val:  0.500440955160933


alpha: 4.3
R2_train:  0.606354200688455
R2_val:  0.500342171290834


alpha: 4.4
R2_train:  0.6061474788314856
R2_val:  0.5002440191513304


alpha: 4.5
R2_train:  0.6059428825106663
R2_val:  0.5001465151504849


alpha: 4.6
R2_train:  0.6057403470645177
R2_val:  0.5000496725955103


alpha: 4.7
R2_train:  0.6055398110668699
R2_val:  0.49995350206851996


alpha: 4.8
R2_train:  0.6053412161011937
R2_val:  0.4998580117550856


alpha: 4.9
R2_train:  0.6051445065546996
R2_val:  0.49976320773210947


alpha: 5.0
R2_train:  0.6049496294301673
R2_val:  0.49966909422052697




In [290]:
## after alp has been decided
alp = 4.4
reg = Ridge(alpha=alp)
reg.fit(X_train_all, y_train_all)

mse_train = np.mean((reg.predict(X_train_all)-y_train_all)**2)
mse_test = np.mean((reg.predict(X_test)-y_test)**2)

R2_train = reg.score(X_train_all, y_train_all)
R2_val = reg.score(X_val, y_val)
R2_test = reg.score(X_test, y_test)
print('R2_train: ', R2_train)
print('R2_val: ', R2_val)
print('R2_test: ', R2_test)
print('\n')

R2_train:  0.5981470409573332
R2_val:  0.584516229248756
R2_test:  0.5856476478639161




In [292]:
imp_feat_inds = np.where(np.abs(reg.coef_)>=40)[0]
imp_feats = [feat_names[k] for k in imp_feat_inds]
imp_feats_score = [reg.coef_[k] for k in imp_feat_inds]
for i in range(len(imp_feats)):
    print(imp_feats[i],':', imp_feats_score[i])

LATITUDE : 46.82951496218378
CCBASIC : -47.86458367500252
PBI : -56.7423193178175
CIP19CERT4 : 54.34970161715748
CIP29CERT4 : 117.81030897057175
CIP30CERT1 : 47.18738338157797
C150_4_2MOR : -51.89637497768749
C150_4_NRA : 47.44036100352819
C150_L4_BLACK : -42.83778338427134
